In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
#visualisation libraries
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use(style="seaborn")

import datetime
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium import plugins
from sklearn.model_selection import TimeSeriesSplit
import warnings
warnings.simplefilter("ignore")
from prophet import Prophet
from statsmodels.graphics.tsaplots import plot_acf

In [3]:
df_Algeria= pd.read_csv("df_Algeria.csv" ,encoding='unicode_escape')
df_Algeria

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,E4InternationalSupport,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality
0,Algeria,2020/1/1,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,Algeria,2020/1/2,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,Algeria,2020/1/3,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,Algeria,2020/1/4,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,Algeria,2020/1/5,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,Algeria,2021/9/24,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028442
633,Algeria,2021/9/25,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028469
634,Algeria,2021/9/26,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028497
635,Algeria,2021/9/27,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028520


In [4]:
#shift 21days mortality
df_Algeria_mortality=df_Algeria["mortality"]
df_Algeria_mortality.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])

22     0.000000
23     0.000000
24     0.000000
25     0.000000
26     0.000000
         ...   
632    0.028442
633    0.028469
634    0.028497
635    0.028520
636    0.000000
Name: mortality, Length: 615, dtype: float64

In [5]:
#删除后21天
df_Algeria.drop(df_Algeria.index[616:637], inplace=True)
df_Algeria

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,E4InternationalSupport,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality
0,Algeria,2020/1/1,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,Algeria,2020/1/2,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,Algeria,2020/1/3,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,Algeria,2020/1/4,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,Algeria,2020/1/5,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,Algeria,2021/9/3,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,2.0,2.0,0.0,0.0,3.0,3.0,2.0,0.027232
612,Algeria,2021/9/4,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,2.0,2.0,0.0,0.0,3.0,3.0,2.0,0.027315
613,Algeria,2021/9/5,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,2.0,2.0,0.0,0.0,3.0,3.0,2.0,0.027373
614,Algeria,2021/9/6,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,2.0,2.0,0.0,0.0,3.0,3.0,2.0,0.027457


In [6]:
df_Algeria1=df_Algeria.join(df_Algeria_mortality,how="left",rsuffix="after21days")
df_Algeria1

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality,mortalityafter21days
0,Algeria,2020/1/1,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,Algeria,2020/1/2,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,Algeria,2020/1/3,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,Algeria,2020/1/4,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4,Algeria,2020/1/5,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,Algeria,2021/9/3,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,2.0,0.0,0.0,3.0,3.0,2.0,0.027232,0.027232
612,Algeria,2021/9/4,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,2.0,0.0,0.0,3.0,3.0,2.0,0.027315,0.027315
613,Algeria,2021/9/5,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,2.0,0.0,0.0,3.0,3.0,2.0,0.027373,0.027373
614,Algeria,2021/9/6,43851044,73%,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,2.0,0.0,0.0,3.0,3.0,2.0,0.027457,0.027457


In [7]:
df_Algeria1.to_csv("df_Algeria_after21days",index=False)